In [ ]:
from os import mkdir, path
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt 
from matplotlib import figure, use

In [ ]:
path_sun_radiaton = "sun_radiation_specific.csv"

path_panels = ["Panels_level7", 
               "Panels_level6", 
               "Panels_level5", 
               "Panels_level4",
               "Panels_level3",
               "Panels_level2",
               "Panels_level1"]

In [ ]:
df_sun_radiation = pd.read_csv(path_sun_radiaton, delimiter=';')

In [ ]:
columns_hours = ["0_1","1_2","2_3","3_4","4_5","5_6","6_7","7_8","8_9","9_10","10_11","11_12","12_13","13_14","14_15","15_16","16_17","17_18","18_19","19_20","20_21","21_22","22_23","23_24"]   

In [ ]:
list_zones = ['A1', 'A3', 'A2', 'A5', 'A6', 'A7', 'A4', 'B5', 'B4', 'B3', 'B2','B1']

In [ ]:
def prepare_panels(raw_df: pd.DataFrame, zone:str):
    new_df = raw_df.drop(['K', 'x', 'y', 'z'], axis=1)
    new_df = new_df.groupby(['spaceName', 'orient', 'name', 'zone'])['area'].sum().reset_index(name ='area_wall_sum')
    new_df = new_df.drop(new_df[new_df.zone != zone].index )

    return new_df

In [ ]:
def prepare_sun_rad_for_special_zone(raw_sun_df: pd.DataFrame,
                                     df_zone_sorted: pd.DataFrame):

    not_drop_list_panels = []
    not_drop_list_orient = []

    for index, row in df_zone_sorted.iterrows():
        not_drop_list_panels.append(row['name'])
        not_drop_list_orient.append(row['orient'])

    new_sun_df = raw_sun_df.loc[raw_sun_df['name'].isin(not_drop_list_panels)]
    new_sun_df = new_sun_df.loc[new_sun_df['orient'].isin(not_drop_list_orient)]

    return new_sun_df

In [ ]:
def merge_panels_and_sunrad(df_panels: pd.DataFrame, df_sun_rad: pd.DataFrame):
    
    new_df = pd.merge(df_panels, df_sun_rad,  how='left', left_on=['name','orient'], right_on = ['name','orient'])
    new_df = new_df.dropna()

    return new_df   

In [ ]:
def calculate_heatsurplus(merged_df: pd.DataFrame, columns_hours: list):
    
    for column_name in columns_hours:  
        merged_df[f'{column_name}_W'] = round(merged_df.area_wall_sum * merged_df[column_name])
    merged_df.loc['Total'] = merged_df.sum(numeric_only=True)

    return merged_df

In [ ]:
def find_max_hour(df: pd.DataFrame) -> str:

    df_max_value = df.drop(['spaceName', 'orient', 'name', 'zone', 'area_wall_sum'], axis=1)
    df_max_value.drop(columns_hours, axis=1, inplace=True)

    max_heat_surplus_W = df_max_value.max(axis=0)
    
    return  max_heat_surplus_W.idxmax()

In [ ]:
def prepare_df_to_plot(df: pd.DataFrame):

    new_col = []
    for index, row in df.iterrows():
        
        if type(row['zone']) != str:
            new_col.append('Total')
        else:
            new_col.append(str(row['zone'])+"_"+str(row['spaceName'])+"_"+str(row['name'])+"_"+str(row['orient']))

    new_df = df.drop([i for i in df.columns if "W" not in i], axis=1)

    new_df['name'] = new_col
    new_df = new_df.set_index('name', drop=True)
    new_df = new_df.T
    
    return new_df

In [ ]:
def plot_df(df: pd.DataFrame, name_output: str, y_limit:int, y_step:int):

    plt.ioff()

    # Create a new figure, plot into it, then close it so it never gets displayed
    fig = plt.figure()

    plt.figure(figsize=(24, 12), dpi=80)

    plt.xticks([i for i in range(24)])
    plt.yticks([i for i in range(0, y_limit, y_step)])

    # plt.plot(df.index, df['Total'], "-", label='Теплопоступления сумма, Вт')

    for column in df.columns:
        plt.plot(df.index, df[column], "-", label=f'Теплопоступления {column}, Вт')


    plt.title(f"Теплопоступления в зоне по часам - {name_output}")
    plt.xlabel('Часы суток')
    plt.ylabel('Теплопоступления, Вт')
    plt.grid(True)
    plt.legend()
    plt.savefig(name_output)
    plt.close(fig)


In [ ]:
def calculate_zone(raw_df_panels: pd.DataFrame, 
                   input_zone:str, 
                   df_sun_radiation: pd.DataFrame, 
                   columns_hours:list, 
                   output_csv_name:str
                   ):
    
    df_panels_prepared = prepare_panels(raw_df_panels, input_zone)
    df_sun_specific = prepare_sun_rad_for_special_zone(df_sun_radiation, df_panels_prepared)
    merged_df = merge_panels_and_sunrad(df_panels_prepared, df_sun_specific)
    new_df = calculate_heatsurplus(merged_df, columns_hours)
    max_hour = find_max_hour(new_df).replace('_W', "")

    columns_for_drop = columns_hours[::]
    columns_for_drop.remove(max_hour)

    new_df.drop(columns_for_drop, axis=1, inplace=True)
    new_df.index.name = 'id'

    new_df.to_csv(output_csv_name)

    return new_df

In [ ]:
df_zone_all_levels = pd.DataFrame()

for level_path in path_panels:
    
    raw_df_panels = pd.read_csv(level_path +".txt", names=["id", "name", "x", "y", "z", "zone", "spaceName", "area", "K", "orient"])

    temp_df = raw_df_panels.copy()
    temp_df['zone'] = temp_df['zone'].str.replace(" Фонарь", "")
    temp_df['zone'] = temp_df['zone'].str.replace("Атриум ", "")

    df_zone_full_level = pd.DataFrame()

    for zone_name in list_zones:
        
        if not path.isdir(f"zones --- {level_path}"):
            mkdir(f"zones --- {level_path}", mode=0o777)

        new_df = calculate_zone(temp_df, 
                    zone_name, 
                    df_sun_radiation, 
                    columns_hours, 
                    f"zones --- {level_path}/{level_path}_zone{zone_name}.csv")

        prepared_df = prepare_df_to_plot(new_df)

        plot_df(prepared_df, f"zones --- {level_path}/{level_path}_zone{zone_name}.png", 300000, 5000)


        # сборка df для общей нагрузки на зону
        df_zone_all_levels[f'{level_path} - {zone_name}'] = prepared_df['Total']


        # сборка df для общей нагрузки на этаж
        df_zone_full_level[f'{level_path} - {zone_name}'] = prepared_df['Total']


        # нагрузка на зону суммарно 
        if not path.isdir(f"summary_by_zone --- {zone_name}"):
            mkdir(f"summary_by_zone --- {zone_name}", mode=0o777)


    # нагрузка на этаж суммарно 
    if not path.isdir(f"summary_by_level --- {level_path}"):
        mkdir(f"summary_by_level --- {level_path}", mode=0o777)
    
    df_zone_full_level = df_zone_full_level.T
    df_zone_full_level.loc['Total'] = df_zone_full_level.sum(numeric_only=True)
    df_zone_full_level = df_zone_full_level.T
    
    df_zone_full_level.to_csv(f'summary_by_level --- {level_path}/df_zone_full_level_{level_path}.csv')
    plot_df(df_zone_full_level, f"summary_by_level --- {level_path}/df_zone_full_level_{level_path}.png", 10000000, 100000)
        

In [ ]:
#нагрузка на здание суммарно по часам
df_all_building = pd.DataFrame()

# нагрузка на зоны суммарно 
for zone_name in list_zones:
    
    df_zone = df_zone_all_levels.copy()

    df_zone = df_zone.drop([i for i in df_zone.columns if zone_name not in i], axis=1)
    
    df_zone = df_zone.T
    df_zone.loc['Total'] = df_zone.sum(numeric_only=True)
    df_zone = df_zone.T

    df_zone.to_csv(f'summary_by_zone --- {zone_name}/summary_by_zone --- {zone_name}.csv')

    plot_df(df_zone, f"summary_by_zone --- {zone_name}/summary_by_zone --- {zone_name}.png", 10000000, 100000)


    if not path.isdir("full_building"):
        mkdir("full_building", mode=0o777)

    df_all_building[zone_name] = df_zone['Total']


df_all_building = df_all_building.T
df_all_building.loc['Total'] = df_all_building.sum(numeric_only=True)
df_all_building = df_all_building.T

df_all_building.to_csv("full_building/full_building.csv")
plot_df(df_all_building, "full_building/full_building.png", 10000000, 100000)

